# Function 4 - Warehouse Optimisation

### First Inspection

According to the description: Address the challenge of optimally placing products across warehouses for a business with high online sales, where accurate calculations are costly and only feasible biweekly. To speed up decision-making, an ML model $f(\mathbf{x})$ approximates these results within hours. The model has four hyperparameters, $x_1, x_2. x_3$ and $x_4$ to tune, and its output $y$ reflects the difference from the expensive baseline. Because the system is dynamic and full of local optima, it requires careful tuning and robust validation to find reliable, near-optimal solutions. 

Let us load and inspect the evalutations,

In [13]:
# Depedencies,
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# SKOPT imports,
from skopt import gp_minimize
from skopt.space import Real
from skopt.learning import GaussianProcessRegressor
from skopt.learning.gaussian_process.kernels import Matern, ConstantKernel, WhiteKernel
from skopt import Optimizer
from joblib import dump, load

# Loading known evaluations,
X, y = np.load("initial_inputs.npy"), np.load("initial_outputs.npy")

# Inspecting evalutation points,
X

array([[0.89698105, 0.72562797, 0.17540431, 0.70169437],
       [0.8893564 , 0.49958786, 0.53926886, 0.50878344],
       [0.25094624, 0.03369313, 0.14538002, 0.49493242],
       [0.34696206, 0.0062504 , 0.76056361, 0.61302356],
       [0.12487118, 0.12977019, 0.38440048, 0.2870761 ],
       [0.80130271, 0.50023109, 0.70664456, 0.19510284],
       [0.24770826, 0.06044543, 0.04218635, 0.44132425],
       [0.74670224, 0.7570915 , 0.36935306, 0.20656628],
       [0.40066503, 0.07257425, 0.88676825, 0.24384229],
       [0.6260706 , 0.58675126, 0.43880578, 0.77885769],
       [0.95713529, 0.59764438, 0.76611385, 0.77620991],
       [0.73281243, 0.14524998, 0.47681272, 0.13336573],
       [0.65511548, 0.07239183, 0.68715175, 0.08151656],
       [0.21973443, 0.83203134, 0.48286416, 0.08256923],
       [0.48859419, 0.2119651 , 0.93917791, 0.37619173],
       [0.16713049, 0.87655456, 0.21723954, 0.95980098],
       [0.21691119, 0.16608583, 0.24137226, 0.77006248],
       [0.38748784, 0.80453226,

Despite the sparsity of data points, the domain of the black box function seems to be $[0, 1]^4$.

In [35]:
# Inspecting black-box outputs,
y

array([-22.10828779, -14.60139663, -11.69993246, -16.05376511,
       -10.06963343, -15.48708254, -12.68168498, -16.02639977,
       -17.04923465, -12.74176599, -27.31639636, -13.52764887,
       -16.6791152 , -16.50715856, -17.81799934, -26.56182083,
       -12.75832422, -19.44155762, -28.90327367, -13.70274694,
       -29.4270914 , -11.56574199, -26.85778644,  -7.96677535,
        -6.70208925, -32.62566022, -19.98949793,  -4.02554228,
       -13.12278233, -23.1394284 ])

The output of the black-box function seem to always be negative and of similar magnitude. We want to maximise $f\mathbf(x)$ so that we can minimise the difference from the expensive baseline. Our data points are too sparse to come up with a conclusion on the smoothness of the function.

### Optimiser Configuration

From the discription of the black-box function $f(\mathbf{x})$, we know,

- It has many local maxima. 
- Unlikely to be noisy (problem discription has not mentioned noise).

However, we do NOT know,

- If $f(\mathbf{x})$ is very smooth, generally smooth or rough.

Again since we do not know the smoothness of $f(\mathbf{x})$, we opt for the Matérn 5/2 kernel (with ARD) as it provides a flexible prior that does not enforce excessive smoothness. LCB with a decaying expoitation/exploration trade-off parameter $\kappa$ is choosen as the acquistion function. Initially, a large value of 
$\kappa$ encourages exploration, allowing the optimiser to sample broadly across the input space and identify multiple promising regions corresponding to different local maxima. As $\kappa$ decays over iterations, the optimisation strategy progressively shifts toward exploitation. Given the multimodal nature of f(x), this design choice intentionally increases the likelihood that the optimiser converges to and exploits a high-quality local maximum rather than continuing extensive global exploration in pursuit of the global optimum. This approach reflects a deliberate trade-off between exploration and exploitation under a limited evaluation budget, prioritising reliable convergence to a strong local solution.

Our initial kappa, is set to $\kappa_0=5.0$ (high amount of exploration) and decays exponentially in accordance to,

$$
\kappa(n) = \kappa_0 e^{-\frac{n}{\tau}}
$$

where $n$ is the number of interactions and $\tau$ is the reciprocal of our decay rate. We set $\tau = 3.75$ such that $\kappa(n) > 1$ (exploitation favoured) when $n > 6$ (we have a total of 12 evaluations). For completion, the elements of our kernel $\mathbf{K}$ are given by,

$$
K(\mathbf{x}_i, \mathbf{x}_j)
=
\sigma^2
\frac{2^{1-\nu}}{\Gamma(\nu)}
\left(
\sqrt{2\nu}\, r_{ij}
\right)^{\nu}
K_{\nu}
\left(
\sqrt{2\nu}\, r_{ij}
\right),
$$

$$
r_{ij}
=
\sqrt{
\frac{(x_{i1} - x_{j1})^2}{\ell_1^2}
+
\frac{(x_{i2} - x_{j2})^2}{\ell_2^2}
+
\frac{(x_{i3} - x_{j3})^2}{\ell_3^2}
+
\frac{(x_{i4} - x_{j4})^2}{\ell_4^2}
}.
$$

where we have set the lengthscales as $\ell_1 = \ell_2 = \ell_3 = \ell_4 = 0.1$ and used $\nu = 5/2$ as well as $\sigma = 1$.

### Optimiser Initialisation

In [36]:
"""INITALISING THE OPTIMISATION MODEL."""

# Inputting the given evaluations provided by the problem,
X_supplied = X.tolist()
y_supplied = y.tolist()

# Inital kappa value,
initial_kappa = 5

"""OPTIMISER SETTINGS."""

# We define the domain of the black-box function (or the range of the parameter values we want to consider),
space = [Real(0, 1, name="x1"),
         Real(0, 1, name="x2"),
         Real(0, 1, name="x3"),
         Real(0, 1, name="x4")
         ]

# Creating the kernel for the GPR,
kernel = ConstantKernel(1.0) * Matern(
    length_scale=(0.1, 0.1, 0.1, 0.1),
    length_scale_bounds=(1e-2, 1.0),
    nu = 5/2)

# GPR settings,
gpr = GaussianProcessRegressor(
    kernel=kernel,
    normalize_y=True,
    n_restarts_optimizer=10
)

# Creating optimisier,
opt = Optimizer(
    dimensions=space,
    base_estimator=gpr,
    acq_func="LCB",
    acq_func_kwargs={"kappa": initial_kappa},
    random_state=0
)

"""CREATING INTIAL OPTIMISER STATE."""

# Supplying given points to optimiser,
opt.tell(X_supplied, (-np.array(y_supplied)).tolist()) # <-- We flip the values since we are trying to maximise the black-box function.

# Asking for the next point to evaluate the black-box function,
point_query = opt.ask()

# Saving optimiser state (zero-th iteration),
dump(opt, "bayes_opt_state_iter0.joblib")

# Printing point query,
print(f"Point Query: {point_query}")

Point Query: [0.0, 0.0, 1.0, 1.0]


### Next Query 

In [17]:
def kappa_decay(init_kappa, decay_param, iter_num):
    """Outputs a value of kappa given the current interaction number (current query number)."""
    return init_kappa*np.exp(-(iter_num/decay_param))

In [ ]:
current_query = 1

# Input the new evaluation,
X_new = [[]]
y_new = []

# Loading the previous state of the optimiser,
opt = load(f"bayes_opt_state_iter{current_query - 1}.joblib")

# Updating kappa,
new_kappa = kappa_decay(init_kappa=initial_kappa, decay_param=3.75, iter_num=current_query)
opt.acq_func_kwargs["kappa"] = new_kappa

# Supplying the new query to the optimiser,
opt.tell(X_new, (-np.array(y_new)).tolist()) # <-- We flip the values since we are trying to maximise the black-box function.

# Asking for the next point to evaluate the black-box function,
point_query = opt.ask()

# Saving optimiser state,
dump(opt, f"bayes_opt_state_iter{current_query}.joblib")

# Printing point query,
print(f"Point Query {current_query + 1}: {point_query}")